In [1]:
# Standard libraries
import h5py
import copy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interactive

# Append base directory
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
path1p = os.path.dirname(currentdir)
path2p = os.path.dirname(path1p)
libpath = os.path.join(path2p, "lib")

sys.path.insert(0, libpath)
print("Appended library directory", libpath)

# User libraries
from data_io.os_lib import getfiles_walk
from data_io.yaro.yaro_data_read import read_neuro_perf
from signal_lib import resample
from fc.te_idtxl_wrapper import idtxlParallelCPUMulti, idtxlResultsParse
from qt_wrapper import gui_fpath

%load_ext autoreload
%autoreload 2

Appended library directory /home/alfomi/work/mesoscopic-functional-connectivity/codes/lib


## 1. Load Neuronal Data

The neuronal data for a given experiment is given by the 3D matrix of dimensions `[N_Trial, N_Times, N_Channels]`

Notes:
* `DT = 0.05` : the sampling interval is always 50ms
* `N_Times = 201` : the total experiment time is always 10s. All experiments are aligned

In [2]:
# Declare constants
params = {
    "exp_timestep" : 0.05, # 50ms, the standard measurement interval
    "bin_timestep" : 0.2,  # 200ms, the binned interval
    
    # The standard timesteps for different scenarios
    "range_CUE" : (1.0, 1.5),    # 1-1.5 seconds trial time
    "range_TEX" : (2.0, 3.5),    # 2-3.5 seconds trial time
    "range_LIK" : (3.5, 6.0),    # 3.5-6 seconds trial time
    "range_ALL" : (0.0, 10.0),   # 0-10  seconds

#     "samples_window" : "ALL",
    "trial_types" : ["iGO", "iNOGO"],
    "resample"    : {'method' : 'averaging', 'kind' : 'kernel'}  # None if raw data is prefered
#     "subsampling_windows_lenghts" : 4*np.ones((1,51)),  #uniform subsampling, 50 bins

#     "te_delay_stepsize" : 1,
#     "te_receiver_embedding_tau" : 1,
#     "te_sampling_rate" : 20
}

idtxl_settings = {
    'dim_order'       : 'rsp',
    'methods'         : ["BivariateTE", "MultivariateTE"],
    'cmi_estimator'   : 'JidtGaussianCMI',
    'min_lag_sources' : 1,
    'max_lag_sources' : 5
}

In [3]:
# Get paths
root_path = gui_fpath("Path to root folder containing neuronal data", "./")
pathswalk = getfiles_walk(root_path, ['behaviorvar.mat'])
datapaths = {os.path.basename(path) : path for path, file in pathswalk}
pd.DataFrame(datapaths, index=['dirname']).T

,dirname
mtp_15_2018_05_16_a,/mnt/hifo_scratch1/Yaro/data_for_TE/mtp/mtp_15...
mtp_15_2018_05_17_a,/mnt/hifo_scratch1/Yaro/data_for_TE/mtp/mtp_15...
mtp_15_2018_05_18_a,/mnt/hifo_scratch1/Yaro/data_for_TE/mtp/mtp_15...
mtp_15_2018_05_19_a,/mnt/hifo_scratch1/Yaro/data_for_TE/mtp/mtp_15...
mtp_15_2018_05_22_a,/mnt/hifo_scratch1/Yaro/data_for_TE/mtp/mtp_15...
mtp_15_2018_05_23_a,/mnt/hifo_scratch1/Yaro/data_for_TE/mtp/mtp_15...
mtp_15_2018_05_24_a,/mnt/hifo_scratch1/Yaro/data_for_TE/mtp/mtp_15...
mtp_15_2018_05_25_a,/mnt/hifo_scratch1/Yaro/data_for_TE/mtp/mtp_15...
mtp_15_2018_05_29_a,/mnt/hifo_scratch1/Yaro/data_for_TE/mtp/mtp_15...
mtp_15_2018_05_30_a,/mnt/hifo_scratch1/Yaro/data_for_TE/mtp/mtp_15...


In [4]:
out_path  = gui_fpath("Path where to save results", "./")

## Swipe Transfer Entropy via IDTxl

Notation of IDTxl
* `(processes, samples, replications)` equivalent to ours `(channels, times, trials)`
* Thus Yaro matrix `(nTrials, nTimes, nChannels)` is given by `'rsp'`, but
* For example `(nChannels, nTimes)` would be `'ps'`

In [5]:
for folderName, folderPathName in datapaths.items():
    
    #############################
    # Reading and downsampling
    #############################

    # Read LVM file from command line
    data, behaviour = read_neuro_perf(folderPathName)

    # Get parameters
    nTrials, nTimes, nChannels = data.shape
    print("Loaded neuronal data with (nTrials, nTimes, nChannels)=", data.shape)

    assert nTimes == 201, "The number of timesteps must be 201 for all data, got "+str(nTimes)
    

    # Timeline (x-axis)
    tlst = params["exp_timestep"] * np.linspace(0, nTimes, nTimes)

    # Downsample data
    if params["resample"] is not None:
        print("Downsampling from", params["exp_timestep"], "ms to", params["bin_timestep"], "ms")
        params["timestep"] = params["bin_timestep"]
        nTimesDownsampled = int(nTimes * params["exp_timestep"] / params["timestep"])
        tlst_down = params["timestep"] * np.linspace(0, nTimesDownsampled, nTimesDownsampled)
        data_down = np.array([[resample(tlst, data[i, :, j], tlst_down, params["resample"])
                              for i in range(nTrials)] 
                             for j in range(nChannels)])
        
        # Replace old data with subsampled one    
        tlst, data = tlst_down, data_down.transpose(1, 2, 0)
        nTrials, nTimes, nChannels = data.shape
        print("After downsampling data shape is (nTrials, nTimes, nChannels)=", data.shape)
        
#         # Plot downsampling for comparison
#         plt.figure()
#         plt.plot(tlst, data[0, :, 0], label='original')
#         plt.plot(tlst_new, data_new[0, 0, :], '.-', label='downsampled')
#         plt.title("Example before and after downsampling")
#         plt.legend()
#         plt.show()
    else:
        params["timestep"] = params["exp_timestep"]

        
    for trialType in params['trial_types']:
        
        if trialType is None:
            dataEff = data
            fileNameSuffix = ""
        else:
            dataEff = data[behaviour[trialType] - 1]
            fileNameSuffix = "_" + trialType
            print("For trialType =", trialType, "the shape is (nTrials, nTimes, nChannels)=", dataEff.shape)

        #############################
        # Analysis
        #############################

        teWindow = idtxl_settings["max_lag_sources"] + 1

        data_range = list(range(nTimes - teWindow + 1))
        data_lst = [dataEff[:, i:i + teWindow, :] for i in data_range]
        rez = idtxlParallelCPUMulti(data_lst, idtxl_settings, folderName)

        for iMethod, method in enumerate(idtxl_settings['methods']):
            te_data = np.full((3, nChannels, nChannels, nTimes), np.nan)
            
            for iRange in data_range:
                te_data[..., iRange + idtxl_settings["max_lag_sources"]] = np.array(
                    idtxlResultsParse(rez[iMethod][iRange], nChannels, method=method, storage='matrix')
                )

            #######################
            # Save results to file
            #######################
            savename = os.path.join(out_path, folderName + fileNameSuffix + '_' + method + '_swipe' + '.h5')
            print(savename)

            h5f = h5py.File(savename, "w")

            grp_rez = h5f.create_group("results")
            grp_rez['TE_table']    = te_data[0]
            grp_rez['delay_table'] = te_data[1]
            grp_rez['p_table']     = te_data[2]

            h5f.close()

Reading Yaro data from /mnt/hifo_scratch1/Yaro/data_for_TE/mtp/mtp_15/mtp_15_2018_05_16_a
Loaded neuronal data with (nTrials, nTimes, nChannels)= (338, 201, 12)
Downsampling from 0.05 ms to 0.2 ms
After downsampling data shape is (nTrials, nTimes, nChannels)= (338, 50, 12)
For trialType = iGO the shape is (nTrials, nTimes, nChannels)= (154, 50, 12)
Adding data with properties: 12 processes, 6 samples, 154 replications
overwriting existing data
Adding data with properties: 12 processes, 6 samples, 154 replications
overwriting existing data
Adding data with properties: 12 processes, 6 samples, 154 replications
overwriting existing data
Adding data with properties: 12 processes, 6 samples, 154 replications
overwriting existing data
Adding data with properties: 12 processes, 6 samples, 154 replications
overwriting existing data
Adding data with properties: 12 processes, 6 samples, 154 replications
overwriting existing data
Adding data with properties: 12 processes, 6 samples, 154 replicati

testing candidate: (0, 3) maximum statistic, n_perm: 200
testing candidate: (1, 5) maximum statistic, n_perm: 200
testing candidate: (0, 4) maximum statistic, n_perm: 200
testing candidate: (2, 3) maximum statistic, n_perm: 200
testing candidate: (4, 3) maximum statistic, n_perm: 200
testing candidate: (5, 3) maximum statistic, n_perm: 200
testing candidate: (8, 3) maximum statistic, n_perm: 200
testing candidate: (9, 5) maximum statistic, n_perm: 200
testing candidate: (10, 3) maximum statistic, n_perm: 200
testing candidate: (6, 4) maximum statistic, n_perm: 200
testing candidate: (3, 4) maximum statistic, n_perm: 200
testing candidate: (5, 4) maximum statistic, n_perm: 200
testing candidate: (2, 4) maximum statistic, n_perm: 200
testing candidate: (0, 5) maximum statistic, n_perm: 200
testing candidate: (1, 4) maximum statistic, n_perm: 200
testing candidate: (9, 4) maximum statistic, n_perm: 200
 -- not significant

---------------------------- (2) include source candidates
candida

 -- not significant
candidate set current source: [(5, 1), (5, 2), (5, 3), (5, 4), (5, 5)]
testing candidate: (3, 1) maximum statistic, n_perm: 200
testing candidate: (5, 3) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(4, 1), (4, 2), (4, 3), (4, 4), (4, 5)]
testing candidate: (4, 4) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(3, 1), (3, 2), (3, 3), (3, 4), (3, 5)]
testing candidate: (3, 1) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(5, 1), (5, 2), (5, 3), (5, 4), (5, 5)]
testing candidate: (5, 1) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(5, 1), (5, 2), (5, 3), (5, 4), (5, 5)]
testing candidate: (5, 1) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(4, 1), (4, 2), (4, 3), (4, 4), (4, 5)]
testing candidate: (4, 1) maximum statistic, n_perm: 200
 -- not significant
candidate set current source: [(4, 

KeyboardInterrupt: 

candidate set: [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5)]
testing candidate: (11, 1) maximum statistic, n_perm: 200
testing candidate: (0, 1) maximum statistic, n_perm: 200

Target: 1 - testing sources [0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

---------------------------- (1) include target candidates
candidate set: [(1, 1), (1, 2), (1, 3), (1, 4), (1, 5)]

Target: 2 - testing sources [0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11]
testing candidate: (1, 1) maximum statistic, n_perm: 200

---------------------------- (1) include target candidates
candidate set: [(2, 1), (2, 2), (2, 3), (2, 4), (2, 5)]
testing candidate: (2, 1) maximum statistic, n_perm: 200

Target: 6 - testing sources [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11]
Target: 3 - testing sources [0, 1, 2, 4, 5, 6, 7, 8, 9, 10, 11]
Target: 5 - testing sources [0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11]


---------------------------- (1) include target candidates

---------------------------- (1) include target candidates
Target: 4 - testing sources [0, 1, 2, 3

testing candidate: (4, 1) maximum statistic, n_perm: 200
